<a href="https://colab.research.google.com/github/rpizarrog/innovacion-empresarial/blob/main/notebook_python/Regresi%C3%B3n_Lineal_Simple_Interactivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión lineal simple

$$
Y = a + b \cdot x
$$

# Proceso:

* Cargar archivo
* Seleccionar variables independiente y dependiente
* Particionar los datos
* Crear el modelo
* Describir interpretación


# Objetivo

Aplicar e interpretar modelos de regresión lineal importando datos dinámicos con variables independiente y dependiente.




# Desarrollo



## Cargar librerías


In [9]:
import io
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

## Cargar funciones


In [11]:

# Paraa variabes globales
df = None  # Declarar df como una variable global inicialmente
df_entrenamiento = None  # Declarar df_entrenamiento como una variable global inicialmente
df_validacion = None  # Declarar df_validacion como una variable global inicialmente
modelo = None  # Declarar modelo como una variable global inicialmente

r_squared = None
coef_a = None
coef_b = None
rmse = None

# Función para cargar el archivo CSV
def cargar_archivo(boton):
    global df  # Acceder a la variable global df
    try:
        nombre_archivo = list(cargar_archivo_selector.value.keys())[0]
        contenido_archivo = cargar_archivo_selector.value[nombre_archivo]['content']
        df = pd.read_csv(io.BytesIO(contenido_archivo))
        # Actualizar las opciones de las variables
        variables_independientes.options = list(df.columns)
        variable_dependiente.options = list(df.columns)
        # Mostrar las primeras y últimas filas del conjunto de datos
        display(df.head(10))
        display(df.tail(10))
    except Exception as e:
        print("Error al cargar el archivo:", e)

# Función para particionar el conjunto de datos
def particionar_entrenamiento_validacion(boton):
    global df, df_entrenamiento, df_validacion  # Acceder a las variables globales df, df_entrenamiento y df_validacion
    try:
        porcentaje = porcentaje_slider.value / 100.0  # Convertir porcentaje a decimal
        if porcentaje <= 0 or porcentaje >= 1:
            print("El porcentaje debe estar entre 0 y 100")
            return
        X = df[variables_independientes.value]
        y = df[variable_dependiente.value]
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=1 - porcentaje, random_state=42)
        # Ordenar los datos de entrenamiento y validación por el índice
        df_entrenamiento = pd.concat([X_train, y_train], axis=1).sort_index()
        df_validacion = pd.concat([X_valid, y_valid], axis=1).sort_index()
        print("Conjunto de datos de entrenamiento ordenado por el índice:")
        display(df_entrenamiento.head(10))
        print("Conjunto de datos de validación ordenado por el índice:")
        display(df_validacion.head(10))
    except Exception as e:
        print("Error al particionar los datos:", e)

# Función para construir el modelo de regresión lineal
def construir_modelo(boton):
    global df, df_entrenamiento, df_validacion, modelo  # Acceder a las variables globales df, df_entrenamiento, df_validacion y modelo
    try:
        if df_entrenamiento is None or df_validacion is None:
            print("Primero debe particionar los datos de entrenamiento y validación")
            return

        X_train = df_entrenamiento[variables_independientes.value].values.reshape(-1, 1)
        y_train = df_entrenamiento[variable_dependiente.value]

        modelo = LinearRegression()
        modelo.fit(X_train, y_train)

        # Mostrar la recta de regresión
        plt.scatter(X_train, y_train, color='blue')
        plt.plot(X_train, modelo.predict(X_train), color='red')
        plt.xlabel(variables_independientes.value)
        plt.ylabel(variable_dependiente.value)
        plt.title('Recta de Regresión')
        plt.grid(True)
        plt.show()
        # Calcular R²
        y_pred_train = modelo.predict(X_train)
        r_squared = r2_score(y_train, y_pred_train)
        print("R² (Coeficiente de determinación):", r_squared)
        # Mostrar los coeficientes de la ecuación de regresión
        coef_a = modelo.intercept_
        coef_b = modelo.coef_[0]
        print("Coeficiente 'a':", coef_a)
        print("Coeficiente 'b':", coef_b)
        # Calcular RMSE en los datos de validación
        if df is None:
            print("Primero debe cargar un archivo CSV")
            return
        X_valid = df_validacion[variables_independientes.value].values.reshape(-1, 1)
        y_valid = df_validacion[variable_dependiente.value]
        y_pred_valid = modelo.predict(X_valid)
        rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
        print("RMSE (Root Mean Squared Error):", rmse)

        # Mandar llamar la función directamente
        mostrar_resultados(r_squared, coef_a, coef_b, rmse)

    except Exception as e:
        print("Error al construir el modelo:", e)

# Función para mostrar los resultados del modelo
def mostrar_resultados(r_squared, coef_a, coef_b, rmse):
    # Crear widgets de texto para mostrar los resultados
    descripcion_widget = widgets.HTML(value=f"<b>R² (Coeficiente de determinación):</b> El valor de R² es aproximadamente {r_squared:.2f}, lo que significa que alrededor del {r_squared * 100:.0f}% de la variabilidad en el peso puede ser explicada por la variabilidad en la estatura. Un valor de R² más alto indica que el modelo ajustado explica una mayor proporción de la variabilidad en los datos observados.<br>")
    coef_a_widget = widgets.HTML(value=f"<b>Coeficiente 'a':</b> El coeficiente 'a' es aproximadamente {coef_a:.2f}. Este coeficiente representa la ordenada al origen en la ecuación de regresión lineal. Indica el valor esperado del peso cuando la estatura es cero. Sin embargo, en este contexto, la interpretación de este coeficiente puede no ser significativa ya que es poco probable que haya personas con estatura cero en el conjunto de datos.<br>")
    coef_b_widget = widgets.HTML(value=f"<b>Coeficiente 'b':</b> El coeficiente 'b' es aproximadamente {coef_b:.2f}. Este coeficiente representa la pendiente de la recta de regresión. Indica el cambio esperado en el peso por cada unidad de cambio en la estatura. En este caso, el coeficiente positivo sugiere que a medida que la estatura aumenta, también lo hace el peso.<br>")
    rmse_widget = widgets.HTML(value=f"<b>RMSE (Root Mean Squared Error):</b> El valor del RMSE es aproximadamente {rmse:.2f} kg. El RMSE es una medida de qué tan bien se ajustan las predicciones del modelo a los valores reales de peso. Cuanto menor sea el RMSE, mejor será el ajuste del modelo a los datos observados. En este caso, un RMSE de {rmse:.2f} kg indica que, en promedio, las predicciones del modelo están a unas {rmse:.2f} kg de los valores reales de peso.<br>")

    # Mostrar los widgets
    display(widgets.VBox([descripcion_widget, coef_a_widget, coef_b_widget, rmse_widget]))






## Resultados e Interpretación del modelo


## Crear y ejecutar widgets

Se ejecutan componentes básicos la interfazinterfaz .


In [ ]:
# Crear widgets
cargar_archivo_selector = widgets.FileUpload(accept='.csv')
boton_cargar = widgets.Button(description="Cargar archivo")
boton_cargar.on_click(cargar_archivo)
variables_independientes = widgets.Dropdown(description="Independiente:")
variable_dependiente = widgets.Dropdown(description="Dependiente:")
porcentaje_slider = widgets.IntSlider(description="Porcentaje:", min=50, max=100, step=5, value=70)
boton_particionar = widgets.Button(description="Entrenamiento-Validación")
boton_particionar.on_click(particionar_entrenamiento_validacion)
boton_construir_modelo = widgets.Button(description="Construir modelo")
boton_construir_modelo.on_click(construir_modelo)
# Crear botón para la interpretación
#boton_interpretacion = widgets.Button(description="Interpretación")
#boton_interpretacion.on_click(lambda _: mostrar_resultados(r_squared, coef_a, coef_b, rmse))


# Mostrar widgets
display(cargar_archivo_selector)
display(boton_cargar)
display(variables_independientes)
display(variable_dependiente)
display(porcentaje_slider)
display(boton_particionar)
display(boton_construir_modelo)

# Mostrar widgets
# display(boton_interpretacion)